In [68]:
import pandas as pd
import nltk
import re, random, os
from nltk.corpus import stopwords
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

In [69]:
df = pd.read_csv('iPhone_reviews.csv')

In [70]:
df.shape

(300, 6)

In [71]:
df.head(10)

,Unnamed: 0.1,Unnamed: 0,rating,title,review_text,model
0,0.0,0,5.0 out of 5 stars,5.0 out of 5 stars\nThe best iPhone - perfect ...,\nThis is my first iPhone. After being an Andr...,Apple iPhone 11
1,1.0,1,5.0 out of 5 stars,5.0 out of 5 stars\nComing from Androids plann...,"\nBeen a few weeks since I got the phone, comi...",Apple iPhone 11
2,2.0,2,4.0 out of 5 stars,4.0 out of 5 stars\niPhone 11 review\n,\niPhone11 Review A person who uses every majo...,Apple iPhone 11
3,3.0,3,5.0 out of 5 stars,5.0 out of 5 stars\nNICE PRODUCT\n,\nBEST SELLER SENDING PERFECT PRODUCT\n,Apple iPhone 11
4,4.0,4,5.0 out of 5 stars,5.0 out of 5 stars\nPerfect switch from OnePlu...,\nAfter 5 years of loyalty towards OnePlus an...,Apple iPhone 11
5,5.0,5,5.0 out of 5 stars,5.0 out of 5 stars\nPerfect phone\n,\nIf u read the review you will know that ther...,Apple iPhone 11
6,6.0,6,5.0 out of 5 stars,5.0 out of 5 stars\nA detailed review\n,"\nAfter 3 months of usage, It feels smooth and...",Apple iPhone 11
7,7.0,7,5.0 out of 5 stars,5.0 out of 5 stars\nAwesome\n,\nGood quality\n,Apple iPhone 11
8,8.0,8,5.0 out of 5 stars,5.0 out of 5 stars\nClassic\n,\nWriting review after 2+years.no issues till ...,Apple iPhone 11
9,9.0,9,4.0 out of 5 stars,4.0 out of 5 stars\nBeast with a limp\n,\nBeen almost 10 months I have been using the ...,Apple iPhone 11


In [72]:
import re

def remove_ratings(title):
    return re.sub(r'\d+\.\d+ out of \d+ stars', '', title).strip()

In [73]:
df['title'] = df['title'].apply(remove_ratings)

In [74]:
df['reviews'] =df['review_text']

In [75]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mohdzahan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [76]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/mohdzahan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [77]:
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U00002702-\U000027B0"
    "\U000024C2-\U0001F251"
    "]+",
    flags=re.UNICODE
)

In [78]:

df['reviews'] = df['reviews'].astype(str)

In [79]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def preprocess(text):
    text = text.replace('\n', '.')
    text = emoji_pattern.sub(r'', text)  
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()  
    text = ' '.join(word for word in text.split() if word not in stop_words)  
    return text

df['reviews'] = df['reviews'].apply(preprocess)

In [80]:
nlp = spacy.load('en_core_web_md', disable = ['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    output = []
    for text in texts:
        doc = nlp(text)
        output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return output

tokenized_reviews = lemmatization(df['reviews'].tolist())

In [81]:
tokenized_reviews

[['first',
  'iphone',
  'loyalist',
  'decade',
  'poor',
  'pixel',
  'design',
  'camera',
  'iphone',
  'switch',
  'walled',
  'garden',
  'edge',
  'lcd',
  'screen',
  'low',
  'resolution',
  'unfounded',
  'screen',
  'iphone',
  'clear',
  'good',
  'color',
  'accuracy',
  'punch',
  'lot',
  'research',
  'iphone',
  'pro',
  'max',
  'model',
  'note',
  'oneplus',
  'side',
  'side',
  'iphone',
  'top',
  'camera',
  'department',
  'important',
  'criterion',
  'choose',
  'year',
  'iphone',
  'substantial',
  'update',
  'pro',
  'last',
  'day',
  'excellent',
  'battery',
  'life',
  'day',
  'phone',
  'dual',
  'camera',
  'system',
  'normal',
  'iphone',
  'perfect',
  'compromise',
  'pro',
  'model',
  'primary',
  'sensor',
  'wide',
  'angle',
  'sensor',
  'telephoto',
  'quality',
  'pro',
  'picture',
  'true',
  'life',
  'accurate',
  'color',
  'great',
  'hdr',
  'low',
  'light',
  'performance',
  'top',
  'notch',
  'bionic',
  'butter',
  'smooth'

In [82]:
print(tokenized_reviews)

[['first', 'iphone', 'loyalist', 'decade', 'poor', 'pixel', 'design', 'camera', 'iphone', 'switch', 'walled', 'garden', 'edge', 'lcd', 'screen', 'low', 'resolution', 'unfounded', 'screen', 'iphone', 'clear', 'good', 'color', 'accuracy', 'punch', 'lot', 'research', 'iphone', 'pro', 'max', 'model', 'note', 'oneplus', 'side', 'side', 'iphone', 'top', 'camera', 'department', 'important', 'criterion', 'choose', 'year', 'iphone', 'substantial', 'update', 'pro', 'last', 'day', 'excellent', 'battery', 'life', 'day', 'phone', 'dual', 'camera', 'system', 'normal', 'iphone', 'perfect', 'compromise', 'pro', 'model', 'primary', 'sensor', 'wide', 'angle', 'sensor', 'telephoto', 'quality', 'pro', 'picture', 'true', 'life', 'accurate', 'color', 'great', 'hdr', 'low', 'light', 'performance', 'top', 'notch', 'bionic', 'butter', 'smooth', 'experience', 'lag', 'free', 'ram', 'model', 'note', 'testing', 'promise', 'straight', 'apple', 'underrated', 'misunderstood', 'parameter', 'comparable', 'money', 'andr

In [83]:
final_reviews = [' '.join(review) for review in tokenized_reviews]

In [84]:
final_reviews

['first iphone loyalist decade poor pixel design camera iphone switch walled garden edge lcd screen low resolution unfounded screen iphone clear good color accuracy punch lot research iphone pro max model note oneplus side side iphone top camera department important criterion choose year iphone substantial update pro last day excellent battery life day phone dual camera system normal iphone perfect compromise pro model primary sensor wide angle sensor telephoto quality pro picture true life accurate color great hdr low light performance top notch bionic butter smooth experience lag free ram model note testing promise straight apple underrated misunderstood parameter comparable money android phone late os month year year security update apple year update use financial transaction security release bug fix day critical build quality design unparalleled love design apple premium brand attention detail alignment port element camera cutout gesture poor job samsung dumb implementation note sp

In [85]:
'''criteria = ["battery", "storage", "cost", "performance", "camera", "quality", "value", "appearance"]

for review in tokenized_reviews:
    for criterion in criteria:
        if criterion not in review:
            review.append(criterion)'''

'criteria = ["battery", "storage", "cost", "performance", "camera", "quality", "value", "appearance"]\n\nfor review in tokenized_reviews:\n    for criterion in criteria:\n        if criterion not in review:\n            review.append(criterion)'

In [86]:
dictionary = corpora.Dictionary(tokenized_reviews)

In [87]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(final_reviews)

In [88]:
print(tfidf_matrix.shape)

(300, 2158)


In [89]:
corpus = gensim.matutils.Sparse2Corpus(tfidf_matrix, documents_columns=False)

In [90]:
id2word = dict((v, k) for k, v in tfidf_vectorizer.vocabulary_.items())

In [91]:
LDA = gensim.models.ldamodel.LdaModel

lda_model = LDA(corpus, num_topics=5, random_state=100, chunksize=1000, iterations=500,id2word=id2word,alpha = 'auto',eta='auto')

In [92]:
lda_model.print_topics()

[(0,
  '0.008*"awesome" + 0.006*"medium" + 0.006*"excellent" + 0.006*"phone" + 0.004*"camera" + 0.004*"iphone" + 0.004*"good" + 0.004*"battery" + 0.003*"performance" + 0.003*"amazing"'),
 (1,
  '0.011*"good" + 0.009*"phone" + 0.007*"iphone" + 0.006*"battery" + 0.006*"quality" + 0.005*"nice" + 0.005*"camera" + 0.005*"product" + 0.004*"heating" + 0.004*"day"'),
 (2,
  '0.011*"iphone" + 0.008*"good" + 0.007*"product" + 0.005*"apple" + 0.005*"phone" + 0.005*"use" + 0.004*"camera" + 0.004*"day" + 0.003*"overall" + 0.003*"full"'),
 (3,
  '0.011*"phone" + 0.011*"iphone" + 0.007*"good" + 0.006*"nice" + 0.004*"camera" + 0.004*"price" + 0.004*"battery" + 0.004*"year" + 0.003*"sale" + 0.003*"time"'),
 (4,
  '0.022*"good" + 0.006*"quality" + 0.006*"phone" + 0.005*"product" + 0.005*"iphone" + 0.003*"delivery" + 0.003*"battery" + 0.003*"great" + 0.003*"year" + 0.003*"normal"')]

In [93]:

topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.008*"awesome" + 0.006*"medium" + 0.006*"excellent" + 0.006*"phone" + 0.004*"camera" + 0.004*"iphone" + 0.004*"good" + 0.004*"battery" + 0.003*"performance" + 0.003*"amazing"')
(1, '0.011*"good" + 0.009*"phone" + 0.007*"iphone" + 0.006*"battery" + 0.006*"quality" + 0.005*"nice" + 0.005*"camera" + 0.005*"product" + 0.004*"heating" + 0.004*"day"')
(2, '0.011*"iphone" + 0.008*"good" + 0.007*"product" + 0.005*"apple" + 0.005*"phone" + 0.005*"use" + 0.004*"camera" + 0.004*"day" + 0.003*"overall" + 0.003*"full"')
(3, '0.011*"phone" + 0.011*"iphone" + 0.007*"good" + 0.006*"nice" + 0.004*"camera" + 0.004*"price" + 0.004*"battery" + 0.004*"year" + 0.003*"sale" + 0.003*"time"')
(4, '0.022*"good" + 0.006*"quality" + 0.006*"phone" + 0.005*"product" + 0.005*"iphone" + 0.003*"delivery" + 0.003*"battery" + 0.003*"great" + 0.003*"year" + 0.003*"normal"')


In [94]:
lda_model.show_topics(num_topics=20, num_words=10, log=False, formatted=True)

[(0,
  '0.008*"awesome" + 0.006*"medium" + 0.006*"excellent" + 0.006*"phone" + 0.004*"camera" + 0.004*"iphone" + 0.004*"good" + 0.004*"battery" + 0.003*"performance" + 0.003*"amazing"'),
 (1,
  '0.011*"good" + 0.009*"phone" + 0.007*"iphone" + 0.006*"battery" + 0.006*"quality" + 0.005*"nice" + 0.005*"camera" + 0.005*"product" + 0.004*"heating" + 0.004*"day"'),
 (2,
  '0.011*"iphone" + 0.008*"good" + 0.007*"product" + 0.005*"apple" + 0.005*"phone" + 0.005*"use" + 0.004*"camera" + 0.004*"day" + 0.003*"overall" + 0.003*"full"'),
 (3,
  '0.011*"phone" + 0.011*"iphone" + 0.007*"good" + 0.006*"nice" + 0.004*"camera" + 0.004*"price" + 0.004*"battery" + 0.004*"year" + 0.003*"sale" + 0.003*"time"'),
 (4,
  '0.022*"good" + 0.006*"quality" + 0.006*"phone" + 0.005*"product" + 0.005*"iphone" + 0.003*"delivery" + 0.003*"battery" + 0.003*"great" + 0.003*"year" + 0.003*"normal"')]

In [95]:
topic_list = []

for topic in topics:
    topic_num, topic_words = topic
    words = [word.split('*')[1].strip('"') for word in topic_words.split(" + ")]
    topic_list.append(words)

topic_lda_iphone=[]
[topic_lda_iphone.extend(k) for k in topic_list ]
topic_lda_iphone

['awesome',
 'medium',
 'excellent',
 'phone',
 'camera',
 'iphone',
 'good',
 'battery',
 'performance',
 'amazing',
 'good',
 'phone',
 'iphone',
 'battery',
 'quality',
 'nice',
 'camera',
 'product',
 'heating',
 'day',
 'iphone',
 'good',
 'product',
 'apple',
 'phone',
 'use',
 'camera',
 'day',
 'overall',
 'full',
 'phone',
 'iphone',
 'good',
 'nice',
 'camera',
 'price',
 'battery',
 'year',
 'sale',
 'time',
 'good',
 'quality',
 'phone',
 'product',
 'iphone',
 'delivery',
 'battery',
 'great',
 'year',
 'normal']

In [96]:
def remove_topic(topic_lda_iphone):
    t = ['phone', 'iPhone', 'apple','iphone']
    for i in t:
        while i in topic_lda_iphone:
            topic_lda_iphone.remove(i)
    return topic_lda_iphone

topic_lda_iphone = remove_topic(topic_lda_iphone)
topic_lda_iphone

['awesome',
 'medium',
 'excellent',
 'camera',
 'good',
 'battery',
 'performance',
 'amazing',
 'good',
 'battery',
 'quality',
 'nice',
 'camera',
 'product',
 'heating',
 'day',
 'good',
 'product',
 'use',
 'camera',
 'day',
 'overall',
 'full',
 'good',
 'nice',
 'camera',
 'price',
 'battery',
 'year',
 'sale',
 'time',
 'good',
 'quality',
 'product',
 'delivery',
 'battery',
 'great',
 'year',
 'normal']

In [97]:
len(topic_lda_iphone)

39

In [98]:
coherence_model_lda = CoherenceModel(
    model=lda_model, 
    texts=tokenized_reviews, 
    dictionary=dictionary, 
    coherence='c_v'
)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.7135924337522199
